In [1]:
!pip install optuna
!pip install mlflow
!pip install xgboost
!pip install statsmodels --quiet
!pip install catboost
!pip install statsmodels --quiet
!pip install lightgbm


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from prophet import Prophet
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm
import warnings
import os
import logging

# Config
warnings.filterwarnings('ignore')
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)
logging.getLogger('prophet').setLevel(logging.WARNING)
np.random.seed(42)
tf.random.set_seed(42)

# === SAFETY CONFIGURATION (Chỉnh rule tại đây) ===
SAFETY_CONFIG = {
    'summer_months': [6, 7, 8],
    'floor': {
        'enabled': True,
        'yoy_growth_min': 1.02,    # Ít nhất bằng 102% năm ngoái
        'mom_drop_max': 0.92,      # Không giảm quá 8% so với tháng trước
        'mom_drop_summer': 0.95,   # Mùa hè giảm ít hơn
        'mom_drop_fall': 0.75
    },
    'buffer': {
        'enabled': True,
        'base_sigma': 2.0,         # An toàn cơ bản: 2 Sigma
        'hist_coverage': 1.05,     # Bao phủ 105% lỗi lớn nhất lịch sử
        'summer_coverage': 1.15,   # Mùa hè bao phủ 115%
        'max_cap_pct': 0.50        # Không bơm quá 50% nhu cầu
    }
}

# =============================================================================
# CLASS: WATER DEMAND SAFETY LAYER V2 (Core Logic)
# =============================================================================
class WaterDemandSafetyLayer:
    def __init__(self, config):
        self.config = config
        self.risk_map = {} 

    def fit(self, df_val, y_true, y_pred_raw):
        print("   ⚙️ [Safety Layer] Learning Risk Profiles from Validation...")
        analysis = df_val[['PWSID_enc', 'Month']].copy()
        analysis['Actual'] = y_true.values if hasattr(y_true, 'values') else y_true
        analysis['Pred'] = y_pred_raw
        analysis['Shortage'] = analysis['Actual'] - analysis['Pred']
        
        # 1. Volatility (Độ ổn định)
        risk_profile = analysis.groupby('PWSID_enc')['Shortage'].std().reset_index()
        risk_profile.rename(columns={'Shortage': 'Error_Std'}, inplace=True)
        
        # 2. Max Historical Shortage (Vết sẹo lịch sử)
        shortage_only = analysis[analysis['Shortage'] > 0]
        if not shortage_only.empty:
            max_risk = shortage_only.groupby('PWSID_enc')['Shortage'].max().reset_index()
            max_risk.rename(columns={'Shortage': 'Max_Shortage'}, inplace=True)
        else:
            max_risk = pd.DataFrame(columns=['PWSID_enc', 'Max_Shortage'])
            
        # 3. Max Summer Shortage (Rủi ro mùa hè)
        summer_months = self.config['summer_months']
        summer_data = analysis[analysis['Month'].isin(summer_months)]
        summer_shortage = summer_data[summer_data['Shortage'] > 0]
        if not summer_shortage.empty:
            max_summer_risk = summer_shortage.groupby('PWSID_enc')['Shortage'].max().reset_index()
            max_summer_risk.rename(columns={'Shortage': 'Max_Summer_Shortage'}, inplace=True)
        else:
            max_summer_risk = pd.DataFrame(columns=['PWSID_enc', 'Max_Summer_Shortage'])

        # Merge Profile
        profile = risk_profile.merge(max_risk, on='PWSID_enc', how='left')
        profile = profile.merge(max_summer_risk, on='PWSID_enc', how='left')
        profile = profile.fillna(0)
        self.risk_map = profile.set_index('PWSID_enc').to_dict('index')

    def _apply_guardrails(self, base_pred, df_context):
        """Correction Layer: Smart Floor"""
        if not self.config['floor']['enabled']:
            return base_pred
        
        cfg = self.config['floor']
        lag_12 = df_context['lag_12'].values
        lag_1 = df_context['lag_1'].values
        months = df_context['Month'].values
        
        floor_yoy = lag_12 * cfg['yoy_growth_min']
        floor_mom = lag_1 * cfg['mom_drop_max'] 
        
        summer_mask = np.isin(months, self.config['summer_months'])
        floor_mom[summer_mask] = lag_1[summer_mask] * cfg['mom_drop_summer']
        
        fall_mask = np.isin(months, [9, 10, 11])
        floor_mom[fall_mask] = lag_1[fall_mask] * cfg['mom_drop_fall']
        
        floored_pred = np.maximum.reduce([base_pred, floor_yoy, floor_mom])
        return floored_pred

    def _apply_risk_buffer(self, pred, df_context):
        """Buffer Layer: Adaptive Risk"""
        if not self.config['buffer']['enabled']:
            return pred, np.zeros_like(pred)

        cfg = self.config['buffer']
        pwsid_encs = df_context['PWSID_enc'].values
        months = df_context['Month'].values
        
        buffers = []
        for i, pid in enumerate(pwsid_encs):
            profile = self.risk_map.get(pid, {'Error_Std': 0, 'Max_Shortage': 0, 'Max_Summer_Shortage': 0})
            
            base_buf = cfg['base_sigma'] * profile['Error_Std']
            hist_buf = profile['Max_Shortage'] * cfg['hist_coverage']
            
            summer_buf = 0
            if months[i] in self.config['summer_months']:
                summer_buf = profile['Max_Summer_Shortage'] * cfg['summer_coverage']
            
            final_buf = max(base_buf, hist_buf, summer_buf)
            
            # Cap logic
            if pred[i] > 0:
                cap_val = pred[i] * cfg['max_cap_pct']
                if final_buf > cap_val and final_buf > profile['Max_Shortage'] * 1.1:
                    final_buf = cap_val
            
            buffers.append(final_buf)
            
        return pred + np.array(buffers), np.array(buffers)

    def predict(self, raw_pred, df_context, explain=False):
        # 1. Correction
        floored_pred = self._apply_guardrails(raw_pred, df_context)
        # 2. Risk Buffer
        final_pred, buffer_amount = self._apply_risk_buffer(floored_pred, df_context)
        
        if explain:
            explanation_df = df_context.copy()
            explanation_df['Raw_Model'] = raw_pred
            explanation_df['Floored_Pred'] = floored_pred
            explanation_df['Risk_Buffer'] = buffer_amount
            explanation_df['Final_Forecast'] = final_pred
            return final_pred, explanation_df
        return final_pred

# =============================================================================
# DATA PREP & RIDGE ANCHOR (Common for all models)
# =============================================================================
def load_and_prep_data():
    print("\n📂 Loading & Preprocessing Data...")
    df = pd.read_csv("CaRDS.csv") # <--- REPLACE YOUR FILE PATH
    
    # Simple clean & melt
    df['Variable'] = df['Variable'].astype(str).str.strip().str.lower()
    date_cols = [c for c in df.columns if c not in ['PWSID', 'Variable']]
    df_melt = df.melt(id_vars=['PWSID', 'Variable'], value_vars=date_cols, var_name='Date', value_name='Value')
    df_pivot = df_melt.pivot_table(index=['PWSID', 'Date'], columns='Variable', values='Value').reset_index()
    df_pivot.columns = [c.capitalize() for c in df_pivot.columns]
    df_pivot['Date'] = pd.to_datetime(df_pivot['Date'])
    df = df_pivot.sort_values(['Pwsid', 'Date']).reset_index(drop=True)
    
    # FillNA
    for c in ['Temperature', 'Precipitation', 'Pdsi']:
        if c in df.columns: df[c] = df[c].fillna(df[c].median())
    df['Demand'] = df['Demand'].fillna(0) # Simple fill for demo
    
    # Feature Engineering
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    df['month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
    df['Is_Summer_Peak'] = ((df['Month'] >= 6) & (df['Month'] <= 8)).astype(int)
    
    le = LabelEncoder()
    df['PWSID_enc'] = le.fit_transform(df['Pwsid'])
    
    # Lags (Needed for Smart Floor)
    df['lag_1'] = df.groupby('Pwsid')['Demand'].shift(1)
    df['lag_12'] = df.groupby('Pwsid')['Demand'].shift(12)
    df['diff_12'] = df.groupby('Pwsid')['Demand'].diff(12)
    df['rolling_mean_12'] = df.groupby('Pwsid')['Demand'].transform(lambda x: x.rolling(12, min_periods=1).mean())
    
    if 'Temperature' in df.columns:
        df['CDD'] = np.maximum(df['Temperature'] - 18, 0)
        df['Temp_lag_1'] = df.groupby('Pwsid')['Temperature'].shift(1)
        df['Summer_Heat_Interaction'] = df['Is_Summer_Peak'] * df['CDD']
        
    if 'Precipitation' in df.columns:
        df['Precip_lag_1'] = df.groupby('Pwsid')['Precipitation'].shift(1)

    df = df.fillna(method='bfill').fillna(0)
    
    # Split
    unique_dates = df['Date'].sort_values().unique()
    n_test = int(len(unique_dates) * 0.2)
    n_val = int(len(unique_dates) * 0.15)
    test_start = unique_dates[-n_test]
    val_start = unique_dates[-(n_test + n_val)]
    
    train = df[df['Date'] < val_start].copy()
    val = df[(df['Date'] >= val_start) & (df['Date'] < test_start)].copy()
    test = df[df['Date'] >= test_start].copy()
    
    return train, val, test

# 1. LOAD DATA
train_df, val_df, test_df = load_and_prep_data()

# 2. TRAIN RIDGE ANCHOR (Common Step)
print("\n⚓ Training Ridge Anchor...")
anchor_cols = ['PWSID_enc', 'Year', 'month_sin', 'month_cos', 'Temperature', 'Precipitation']
scaler = StandardScaler()
X_train_anch = scaler.fit_transform(train_df[anchor_cols])
X_val_anch = scaler.transform(val_df[anchor_cols])
X_test_anch = scaler.transform(test_df[anchor_cols])

anchor_model = Ridge(alpha=1.0)
anchor_model.fit(X_train_anch, train_df['Demand'])

train_df['Anchor_Pred'] = anchor_model.predict(X_train_anch)
val_df['Anchor_Pred'] = anchor_model.predict(X_val_anch)
test_df['Anchor_Pred'] = anchor_model.predict(X_test_anch)

# Calculate Residuals (Target for the next model)
train_df['Residuals'] = train_df['Demand'] - train_df['Anchor_Pred']
val_df['Residuals'] = val_df['Demand'] - val_df['Anchor_Pred']
test_df['Residuals'] = test_df['Demand'] - test_df['Anchor_Pred']

print(f"   Ridge Anchor Val R2: {r2_score(val_df['Demand'], val_df['Anchor_Pred']):.4f}")

: 